In [ ]:
import numpy as np
from google.colab import drive
import tensorflow as tf

drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/data/'

train_images = np.load(dataset_path + 'train_images.npy')
train_labels = np.load(dataset_path + 'train_labels.npy')
test_images = np.load(dataset_path + 'test_images.npy')
test_labels = np.load(dataset_path + 'test_labels.npy')

print("Train Images Shape:", train_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Images Shape:", test_images.shape)
print("Test Labels Shape:", test_labels.shape)

print("\nTrain Images Data Type:", train_images.dtype)
print("Train Labels Data Type:", train_labels.dtype)
print("Test Images Data Type:", test_images.dtype)
print("Test Labels Data Type:", test_labels.dtype)

print("\nTrain Images Min Value:", np.min(train_images))
print("Train Images Max Value:", np.max(train_images))
print("Test Images Min Value:", np.min(test_images))
print("Test Images Max Value:", np.max(test_images))

print("\nUnique Train Labels:", np.unique(train_labels))
print("Unique Test Labels:", np.unique(test_labels))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train Images Shape: (28754, 48, 48)
Train Labels Shape: (28754, 7)
Test Images Shape: (7178, 48, 48)
Test Labels Shape: (7178, 7)

Train Images Data Type: float64
Train Labels Data Type: float32
Test Images Data Type: float64
Test Labels Data Type: float32

Train Images Min Value: 0.0
Train Images Max Value: 1.0
Test Images Min Value: 0.0
Test Images Max Value: 1.0

Unique Train Labels: [0. 1.]
Unique Test Labels: [0. 1.]


In [ ]:
#Load the TFLite models
model_path = '/content/drive/MyDrive/models/'
num_models = 5
models = []

for i in range(num_models):
    model_name = f'emotion_detection_model_{i+1}.tflite'
    model_file = model_path + model_name
    interpreter = tf.lite.Interpreter(model_path=model_file)
    interpreter.allocate_tensors()
    models.append(interpreter)

#Get input and output details for the models
input_details = models[0].get_input_details()
output_details = models[0].get_output_details()

#Evaluate the ensemble performance
ensemble_predictions = []

for image in test_images:
    predictions = []
    for model in models:
        # Reshape the input tensor to match the expected shape
        input_shape = input_details[0]['shape']
        input_data = image.reshape(input_shape)

        model.set_tensor(input_details[0]['index'], input_data.astype(np.float32))
        model.invoke()
        prediction = model.get_tensor(output_details[0]['index'])
        predictions.append(prediction)

    ensemble_prediction = np.mean(predictions, axis=0)
    ensemble_predictions.append(np.argmax(ensemble_prediction))

ensemble_predictions = np.array(ensemble_predictions)
print(f"Shape of ensemble_predictions array: {ensemble_predictions.shape}")
accuracy = np.mean(ensemble_predictions == np.argmax(test_labels, axis=1))

print(f"\nEnsemble Accuracy: {accuracy:.4f}")

Shape of ensemble_predictions array: (7178,)

Ensemble Accuracy: 0.6819


In [ ]:
#Load the TFLite models
model_path = '/content/drive/MyDrive/models/'
num_models = 5
models = []

for i in range(num_models):
    model_name = f'emotion_detection_model_{i+1}.tflite'
    model_file = model_path + model_name
    interpreter = tf.lite.Interpreter(model_path=model_file)
    interpreter.allocate_tensors()
    models.append(interpreter)


#Evaluate the ensemble performance
train_ensemble_features = []
test_ensemble_features = []

#Extract features from the training set
for image in train_images:
    train_predictions = []
    for model in models:
        input_shape = input_details[0]['shape']
        input_data = image.reshape(input_shape)
        model.set_tensor(input_details[0]['index'], input_data.astype(np.float32))
        model.invoke()
        prediction = model.get_tensor(output_details[0]['index'])
        train_predictions.append(tuple(prediction.flatten()))
    train_ensemble_features.append(tuple(train_predictions))

#Extract features from the test set
for image in test_images:
    test_predictions = []
    for model in models:
        input_shape = input_details[0]['shape']
        input_data = image.reshape(input_shape)
        model.set_tensor(input_details[0]['index'], input_data.astype(np.float32))
        model.invoke()
        prediction = model.get_tensor(output_details[0]['index'])
        test_predictions.append(tuple(prediction.flatten()))
    test_ensemble_features.append(tuple(test_predictions))



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbdt = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbdt.fit(train_ensemble_features, np.argmax(train_labels, axis=1))

#Evaluate the ensemble performance
ensemble_predictions = gbdt.predict(test_ensemble_features)
accuracy = np.mean(ensemble_predictions == np.argmax(test_labels, axis=1))

print(f"\nEnsemble Accuracy: {accuracy:.4f}")



Ensemble Accuracy: 0.6864
